In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time
from tqdm import tqdm as ProgressBar
import six # needed for Google Cloud client
import operator
import sys
import en # NodeBox https://www.nodebox.net/code/index.php/Linguistics#verb_conjugation
from nltk.corpus import stopwords
import parsing_util
import pickle

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [5]:
df=pd.read_pickle('CleanedIngredients.pkl')
df.drop_duplicates(subset='url', keep='first', inplace=True)
print df.shape
print df[['instructionSteps']].values[0]
print df[['ingredients']].values[0]
print df[['cleanedIngredients']].values[0]


(48417, 11)
[ [u'In a bowl, mix the tomatoes, onion, red bell pepper, yellow bell pepper, cilantro, lime juice, and salt. Cover and refrigerate until ready to serve.']]
[ [u'6 roma (plum) tomatoes, diced', u'1 sweet onion, diced', u'1 medium red bell pepper, diced', u'1 medium yellow bell pepper, diced', u'1 bunch cilantro, finely minced', u'1 lime, juiced', u'1 teaspoon salt, or to taste', u'Add all ingredients to list', u'Add all ingredients to list']]
[ ['roma plum tomatoes', 'sweet onion', 'red bell pepper', 'yellow bell pepper', 'cilantro', 'lime', 'salt']]


In [4]:
# Load a list of stopwords
stopWords = set(stopwords.words('english'))
stopVerbs = {'bring','make'} # These are useless verbs that should be ignored
goVerbs = {'preheat'}  # These are verbs that are often not recognized as such


def get_verbs(parsedInstructions, Ingredients, ingredientsDict, debug=0):

    tokens = parsedInstructions

    # part-of-speech tags from enums.PartOfSpeech.Tag
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')
    idx=0
    verbs=list()
    for token in tokens:
        tag = pos_tag[token.part_of_speech.tag]
        if token.text.content.lower() != token.text.content and en.is_verb(token.text.content.lower()):
            tag = 'VERB'
        # If this was identified and a noun and can be a verb and there is a noun following
        elif (tag == 'NOUN' # parser thinks it is a NOUN but can also be a verb
             and len(tokens) > idx+1
             and en.is_verb(token.text.content.lower()) and pos_tag[tokens[idx+1].part_of_speech.tag] == 'NOUN'):
            if (pos_tag[tokens[idx-1].part_of_speech.tag] == 'PUNCT') and token.text.content not in ingredientsDict:
                tag = 'VERB'
        else:
            tag = pos_tag[token.part_of_speech.tag]

        if pos_tag[token.part_of_speech.tag] == 'VERB':
            verb=(token.text.content).lower()
            #try:        
            #    verb=en.verb.present(verb)
            #except:
            #    pass
            if verb not in stopWords and verb not in stopVerbs:
                verbs.append(verb)
        if pos_tag[token.part_of_speech.tag] == 'VERB':
            verb=(token.text.content).lower()
            if verb in goVerbs:
                verbs.append(verb)
            
        if debug > 5:
            print(u'{}: {}'.format(pos_tag[token.part_of_speech.tag], token.text.content.lower()))
        idx += 1
    return(verbs)

In [6]:
verbsDict=defaultdict(int)
insDict = parsing_util.Instructions()  
cleanedIngredientsDict=pd.read_pickle('cleanerIngredients.pkl')  
added=0
#for idx,row in df.sample(1).iterrows():
for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
    url=row["url"]
    # print url
    parsedInstructions=insDict.parse(url,row["instructionSteps"])
    Ingredients=cleanedIngredientsDict.get(url,[])
    verbs=get_verbs(parsedInstructions, Ingredients, verbsDict, debug=0)
    verbsDict[url] = verbs
    #for verb in verbs:
    #    verbsDict[verb] += 1
        # print verb
    added+=1
    if added > 10000:
        with open('verbs.pkl', 'wb') as f:
            pickle.dump(verbsDict, f) 
        added = 0
    
with open('verbs.pkl', 'wb') as f:
    pickle.dump(verbsDict, f)



Processing recipes: 48417it [03:31, 229.38it/s]


In [8]:
len(allVerbs)

965

In [ ]:
#verbVertor =  [get_verbs('.'.join(df[['instructionSteps']].values[i][0])) for i in range(df.shape[0])]
#df['verbs']=verbVertor